<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PyCaret-tutorial-for-DAT801" data-toc-modified-id="PyCaret-tutorial-for-DAT801-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PyCaret tutorial for DAT801</a></span></li><li><span><a href="#Getting-the-data" data-toc-modified-id="Getting-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Getting the data</a></span></li><li><span><a href="#Last-inn-datasettet-og-ta-en-kikk-på-dets-struktur" data-toc-modified-id="Last-inn-datasettet-og-ta-en-kikk-på-dets-struktur-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Last inn datasettet og ta en kikk på dets struktur</a></span></li><li><span><a href="#Utforsk-data" data-toc-modified-id="Utforsk-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Utforsk data</a></span></li><li><span><a href="#Forbered-data" data-toc-modified-id="Forbered-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Forbered data</a></span></li><li><span><a href="#Tren-noen-baseline-modeller" data-toc-modified-id="Tren-noen-baseline-modeller-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Tren noen baseline-modeller</a></span></li><li><span><a href="#Hyperparameter-tuning" data-toc-modified-id="Hyperparameter-tuning-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Hyperparameter tuning</a></span></li><li><span><a href="#Ensembling" data-toc-modified-id="Ensembling-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Ensembling</a></span></li><li><span><a href="#Analyser-resultatene" data-toc-modified-id="Analyser-resultatene-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Analyser resultatene</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Confusion-matrix" data-toc-modified-id="Confusion-matrix-9.0.1"><span class="toc-item-num">9.0.1&nbsp;&nbsp;</span>Confusion matrix</a></span></li><li><span><a href="#Errors" data-toc-modified-id="Errors-9.0.2"><span class="toc-item-num">9.0.2&nbsp;&nbsp;</span>Errors</a></span></li><li><span><a href="#Precision-versus-recall" data-toc-modified-id="Precision-versus-recall-9.0.3"><span class="toc-item-num">9.0.3&nbsp;&nbsp;</span>Precision versus recall</a></span></li><li><span><a href="#Feature-importance" data-toc-modified-id="Feature-importance-9.0.4"><span class="toc-item-num">9.0.4&nbsp;&nbsp;</span>Feature importance</a></span><ul class="toc-item"><li><span><a href="#Tolkning-av-modeller" data-toc-modified-id="Tolkning-av-modeller-9.0.4.1"><span class="toc-item-num">9.0.4.1&nbsp;&nbsp;</span>Tolkning av modeller</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Bruk-modell-på-ny-data" data-toc-modified-id="Bruk-modell-på-ny-data-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Bruk modell på ny data</a></span></li><li><span><a href="#Eksporter-modell" data-toc-modified-id="Eksporter-modell-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Eksporter modell</a></span><ul class="toc-item"><li><span><a href="#Deploy-modell" data-toc-modified-id="Deploy-modell-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Deploy modell</a></span></li></ul></li></ul></div>

**NB: Hvis du vil kjøre denne notebooken på din egen maskin, se installasjonsinstruksjoner for PyCaret på DAT801s GitHub-side**

# PyCaret tutorial for DAT801

<img src="assets/pycaret.png" width="800" height="400">
PyCaret er et open-source maskinlæringsbibliotek i Python som bistår i prosessen fra forbereding av data til deployment. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
if 'google.colab' in str(get_ipython()):
    print('The notebook is running on Colab. colab=True.')
    colab=True
else:
    print('The notebook is not running on Colab. colab=False.')
    colab=False

In [ ]:
if colab:
    !pip install pycaret[all]==2.3.4

# Getting the data

In [ ]:
import pycaret

In [ ]:
from pycaret.datasets import get_data

In [ ]:
_ = get_data('index')

# Last inn datasettet og ta en kikk på dets struktur

For at beregningene ikke skal ta så lang tid bruker vi her et relativt lite datasett. Du er velkommen til å forsøke noen av de andre i listen over! For eksempel `income`.

In [ ]:
#data = get_data('income')
data = get_data('diabetes')

In [ ]:
data['Class variable'].value_counts()

In [ ]:
data.info()

In [ ]:
data.describe()

Target er `Class variable` som er en binær variabel (True / False):

In [ ]:
data['Class variable'].hist()

Vi ser at:
- Det er både numeriske og kategoriske features
- Det er behov for skalering av numeriske features
- Datasettet er ubalansert

# Utforsk data

Etter at en har lastet ned data og undersøkt strukturen bør en sette i gang med en grundigere utforsking. Vi har studert hvordan dette kan gjøres tidligere i kurset. 

Her er en pakke som kan gjøre en god del av den vanlige utforskingen automatisk:

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
ProfileReport(data)

# Forbered data

In [ ]:
from pycaret.classification import *

In [ ]:
data = setup(data=data, target='Class variable', numeric_imputation='median', 
             normalize=True, normalize_method='robust', session_id=42)

# Tren noen baseline-modeller

In [ ]:
models()

In [ ]:
top_models = compare_models(n_select=6, sort='F1')

# Hyperparameter tuning

Nå har vi funnet noen kandidat-modeller: 

In [ ]:
top_models

Vi tune deres hyperparametre for å forsøke å forbedre deres ytelse (merk at vi kjører litt ulike antall iterasjoner da noen av modellene er ganske trege å trene):

In [ ]:
tuned_ada = tune_model(top_models[0], fold=10, n_iter=100, optimize='F1')

In [ ]:
tuned_lightgbm = tune_model(top_models[1], fold=10, n_iter=400, optimize='F1')

In [ ]:
tuned_gbc = tune_model(top_models[2], fold=10, n_iter=100, optimize='F1')

In [ ]:
tuned_catboost = tune_model(top_models[3], fold=10, n_iter=50, optimize='F1')

In [ ]:
tuned_rf = tune_model(top_models[4], fold=10, n_iter=300, optimize='F1')

> Merk at vi har brukt default parameter-grids satt av PyCaret. Ofte kan det være lurt å se nøyere på valg av grid en søker over. 

Her er de fire beste modellene og de beste hyperparametrene vi fant for hver:

Vi kan nå score disse oppdaterte modellene:

In [ ]:
compare_models(include=[tuned_ada, tuned_lightgbm, tuned_gbc, tuned_catboost, tuned_rf], sort='F1')

Disse scores kan sammenlignes med det vi fant over da vi brukte default-parametre. Merk at `tune_model` bruker `RandomizedSearchCV` fra scikit-learn som default-søk. Det er derfor ikke sikkert at vi finner bedre hyperparametre enn default i løpet av søket. 

# Ensembling

Som vi har sett kan en ofte kombinere modeller slik at kombinasjonen blir bedre enn hver enkelt modell. Vi har fokusert på "voting ensembles", men det finnes en rekke måter å gjøre dette på. I PyCaret kan vi bruke `blend_models` til å konstruere voting ensembles. La oss bruke de tre beste modellene etter tuning:

In [ ]:
best_models = [tuned_rf, tuned_lightgbm, tuned_gbc]

In [ ]:
voting_soft = blend_models(best_models, method='soft', optimize='F1')

In [ ]:
voting_hard = blend_models(best_models, method='hard', optimize='F1')

Som nevnt kan en også trene en såkalt "blender" på toppen av prediksjonene til et sett med modeller, for slik å kunne anvende mer kompliserte sammenhenger enn i et voting ensemble:

In [ ]:
blender = stack_models(estimator_list=best_models, optimize='F1')

I dette tilfellet var det ingen av disse ensembles som var bedre enn den beste enkeltmodellen.

# Analyser resultatene

In [ ]:
%matplotlib inline

In [ ]:
model = voting_hard
model

### Confusion matrix

Her er forvirringsmatrisen på de 9769 instansene i test-settet:

In [ ]:
plot_model(model, 'confusion_matrix')

### Errors

Her er et plot av hvilke feil som blir gjort:

In [ ]:
plot_model(model, 'error')

### Precision versus recall

Som vi har diskutert er det en tradeoff mellom precision og recall ("bias-variance tradeoff"). Vi kan visualisere hvor treshold har blitt satt av vår modell:

In [ ]:
plot_model(best_models[0], 'threshold')

Hvis en ønsker å endre på dette tresholdet (f.eks. hvis falske positive er verre enn falske negative) kan en bruke metoden `optimize_threshold`.

### Feature importance

Hvilke features er det modellene lener seg mest på?

In [ ]:
best_models

In [ ]:
print(best_models[0])

In [ ]:
plot_model(best_models[0], 'feature')

In [ ]:
print(best_models[1])

In [ ]:
plot_model(best_models[1], 'feature')

In [ ]:
print(best_models[2])

In [ ]:
plot_model(best_models[2], 'feature')

#### Tolkning av modeller

Vi kan gå litt dypere inn i hvordan modellene fungerer, for eksempel via [Shapley values](https://christophm.github.io/interpretable-ml-book/shapley.html). 

In [ ]:
interpret_model(best_models[0], plot='correlation')

In [ ]:
interpret_model(best_models[0])

# Bruk modell på ny data

In [ ]:
# Prediker på test-settet satt til side tidligere
y_pred = predict_model(model)

In [ ]:
y_pred.head()

# Eksporter modell

Når en er ferdig med å konstruere, trene, evaluere og forstå modellene er det tid for å sette modellen ut i drift. En ønsker da å eksportere modellen sammen med hele pipeline for preprosessering til disk eller til en skytjeneste. 

Om man skal predikere på helt ny data (dvs. er ferdig med modell-byggingen) så kan man bruke `predict_model` også på denne. Da vil data preprosesseres og kjøres gjennom modellen. 

MEN: Hittil har vi satt til side data til test. Når en er ferdig med modell-byggingen så er det ikke noe vits å sette data til side. En bør trene modellen på alt en har av labelet data. 

Dette kan gjøres med `finalize_model`:

In [ ]:
final_model = finalize_model(model)

Deretter kan vi lagre modellen:

In [ ]:
save_model(final_model,'saved_model')

## Deploy modell

PyCaret kan hjelpe med deployment til AWS (https://pycaret.org/deploy-model/), men her er det så mange valgmuligheter at det er vanskelig å ha én oppskrift for alt. 